In [0]:
!pip install advertorch > /dev/null
from __future__ import print_function
import os
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable 
from advertorch.attacks import GradientSignAttack, LinfBasicIterativeAttack
import numpy as np

In [0]:
import sys 
import torchvision.datasets as datasets 
import torchvision.transforms as transforms 
from torch.utils.data.sampler import SubsetRandomSampler as SRS 
import torch.utils.data as data_utils 
import numpy as np 
CIFAR10_DATA_ROOT = './cifar10_data/'
SEED = 161803398 # Golden Ratio
def get_cifar10_data_loaders(batch_size=64, n_train=40000, \
    n_val=10000, n_test=10000, train_transform=None, \
    val_transform=None, test_transform=None):

    assert n_train + n_val == 50000
    if train_transform is None:
        train_transform = transforms.Compose([
            transforms.RandomCrop(size=32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])
    if val_transform is None:
        val_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])
    if test_transform is None:
        test_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])
    
    train_set = datasets.CIFAR10(root=CIFAR10_DATA_ROOT, download=True, \
        train=True, transform=train_transform)
    val_set = datasets.CIFAR10(root=CIFAR10_DATA_ROOT, download=True, \
        train=True, transform=val_transform)
    test_set = datasets.CIFAR10(root=CIFAR10_DATA_ROOT, download=True, \
        train=False, transform=test_transform)

    indices = np.arange(0, 50000)
    np.random.seed(SEED)
    np.random.shuffle(indices)

    train_sampler = SRS(indices[:n_train])
    val_sampler = SRS(indices[n_train:])
    test_sampler = SRS(np.arange(0, 10000))

    train_loader = data_utils.DataLoader(train_set, batch_size=batch_size, \
        sampler=train_sampler)
    val_loader = data_utils.DataLoader(val_set, batch_size=batch_size, \
        sampler=val_sampler)
    test_loader = data_utils.DataLoader(test_set, batch_size=batch_size, \
        sampler=test_sampler)
    
    return train_loader, val_loader, test_loader 

def progress(curr, total, suffix='', bar_len=48):
    filled = int(round(bar_len * curr / float(total))) if curr != 0 else 1
    bar = '=' * (filled - 1) + '>' + '-' * (bar_len - filled)
    sys.stdout.write('\r[%s](%d/%d) .. %s' % (bar, curr, total, suffix))
    sys.stdout.flush()
    if curr == total:
        bar = bar_len * '='
        sys.stdout.write('\r[%s](%d/%d) .. %s .. Completed\n' % (bar, curr, total, suffix))
    return 

def _preprocess_state_dict(state_dict):
    from collections import OrderedDict
    new_dict = OrderedDict()
    for k, v in state_dict.items():
        if not k.endswith('num_batches_tracked'):
            if k.startswith('module.'):
                new_dict[k[7:]] = v
            else:
                new_dict[k] = v
    return new_dict

In [3]:
!wget https://github.com/aamir-mustafa/pcl-adversarial-defense/raw/master/robust_model.pth.tar

--2020-01-31 13:17:49--  https://github.com/aamir-mustafa/pcl-adversarial-defense/raw/master/robust_model.pth.tar
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aamir-mustafa/pcl-adversarial-defense/master/robust_model.pth.tar [following]
--2020-01-31 13:17:49--  https://raw.githubusercontent.com/aamir-mustafa/pcl-adversarial-defense/master/robust_model.pth.tar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16162240 (15M) [application/octet-stream]
Saving to: ‘robust_model.pth.tar.1’

robust_model.pth.ta 100%[===================>]  15.41M  --.-KB/s    in 0.08s   

2020-01-31 13:17:50 (199 MB/s) - ‘

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Kanged from https://raw.githubusercontent.com/aamir-mustafa/pcl-adversarial-defense/master/resnet_model.py
"""
Created on Tue Apr  2 14:21:30 2019

@author: aamir-mustafa
"""

import torch.nn as nn
import math

def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):  # (conv-bn-relu) x 3 times
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual   # in our case is none
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, depth, num_classes=10):
        super(ResNet, self).__init__()
        # Model type specifies number of layers for CIFAR-10 model
        assert (depth - 2) % 6 == 0, 'depth should be 6n+2'
        n = (depth - 2) // 6

        block = Bottleneck if depth >=44 else BasicBlock

        self.inplanes = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 16, n)
        
        self.layer2 = self._make_layer(block, 32, n, stride=2)
        self.layer3 = self._make_layer(block, 64, n, stride=2)
        self.avgpool = nn.AvgPool2d(8)
        
        self.maxpool2= nn.MaxPool2d(16)
        self.fc = nn.Linear(64 * block.expansion, 1024)
        self.fcf = nn.Linear(1024,num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        

        x = self.bn1(x)
        x = self.relu(x)   

        x = self.layer1(x)  
        x = self.layer2(x)  
        
        m =  self.maxpool2(x) 
        m = m.view(m.size(0), -1) # 128 dimensional
        
        x = self.layer3(x)  


        x = self.avgpool(x)
        z = x.view(x.size(0), -1) # 256 dimensional
        x = self.fc(z)            # 1024 dimensional
        y = self.fcf(x)           # num_classes dimensional
        
        return y # m, z, x, y - original return values


def resnet(**kwargs):
    """
    Constructs a ResNet model.
    """
    return ResNet(**kwargs)  

In [6]:
model = resnet(num_classes=10, depth=110).cuda()
model.load_state_dict(_preprocess_state_dict(torch.load('./robust_model.pth.tar')['state_dict']))
model.eval()

sub = resnet(num_classes=10, depth=110).cuda()
sub.load_state_dict(_preprocess_state_dict(torch.load('/content/drive/My Drive/reoptimization_attack/cifar_pcl_defense/best_substitute_model_cifar_pcl_defense.pt')))
sub.eval()

adversaries = [
    GradientSignAttack(model, nn.CrossEntropyLoss(size_average=False), eps=float(8.0/255)),
    GradientSignAttack(sub, nn.CrossEntropyLoss(size_average=False), eps=float(8.0/255)),
]
_, _, test_loader = get_cifar10_data_loaders()
for adversary in adversaries:
    correct_adv = 0
    for i, (x_batch, y_batch) in enumerate(test_loader):
        x_batch, y_batch = x_batch.cuda(), y_batch.cuda()
        adv_x_batch = adversary.perturb(x_batch, y_batch)
        logits = model(adv_x_batch)
        _, preds = torch.max(logits, 1)
        correct_adv += (preds == y_batch).sum().item()
        progress(i+1, len(test_loader), 'correct_adv = {}'.format(correct_adv))

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
[================================================](157/157) .. correct_adv = 5842 .. Completed
[================================================](157/157) .. correct_adv = 5761 .. Completed
